## SQL Queries on Washington, DC Airbnb Dataset

Dataset available from http://insideairbnb.com/get-the-data.html. Tables: calendar, listings1, listings2, neighbourhoods, reviews1, reviews2.

### Describe the most expensive listing. 

WITH
	
    average_daily_price
AS(

	SELECT 
		listing_id,
		AVG(CAST(LTRIM(price,'$') as REAL)) as average_daily_rate
	FROM
		calendar
	GROUP BY 1 
)

SELECT

	id,
	neighbourhood,
	zipcode,
	property_type,
	room_type,
	accommodates,
	review_scores_rating,
	summary,
	transit,
	access,
	house_rules,
	average_daily_rate,
	minimum_nights,
	cleaning_fee,
	MAX(((average_daily_rate*CAST(minimum_nights as INTEGER))+CAST(LTRIM(cleaning_fee,'$') as REAL))) as total_cost    
FROM

	listings1    
JOIN

	average_daily_price    
ON

	average_daily_price.listing_id = listings1.id
On a minimum stay basis, the most expensive listing is an apartment that accomodates up to five people in the Mount Pleasant neighborhood, zip code 20009, with a relatively high rating of 90% and fairly lax house rules. At an average daily rate of 233 dollars, minimum stay of 160 nights, and cleaning fee of 200 dollars, the total cost comes up to 37,441 dollars per stay.

#### Alternative:


WITH

	average_daily_price    
AS(

	SELECT 
		listing_id,
		AVG(CAST(LTRIM(price,'$') as REAL)) as average_daily_rate
	FROM
		calendar
	GROUP BY 1
)

SELECT

	id,
	neighbourhood,
	zipcode,
	property_type,
	room_type,
	accommodates,
	review_scores_rating,
	summary,
	transit,
	access,
	house_rules,
	MAX(average_daily_rate) as average_daily_rate,
	minimum_nights,
	cleaning_fee,
	((average_daily_rate*CAST(minimum_nights as INTEGER))+CAST(LTRIM(cleaning_fee,'$') as REAL)) as total_cost    
FROM
	
    listings1
JOIN

	average_daily_price    
ON

	average_daily_price.listing_id = listings1.id
On a daily rate basis, the most expensive listing is an apartment that accomodates up to six people in the Capitol Hill neighborhood, zip code 20003. The entire apartment, which is a newly renovated rowhouse condo, is available for rent and is across the street from Potomac Avenue Metro, walking distance to the U.S. Capitol, Eastern Market and Barracks Row. The only house rule is to treat the home as one's own. At an average daily rate of 994 dollars, minimum of two nights and cleaning fee of 75 dollars, the total cost comes up to 2,062 dollars per stay.

### What neighborhoods seem to be the most popular?

WITH

	availability    
AS(

	SELECT
		listing_id,
		COUNT(listing_id) total_days,
		COUNT(CASE available WHEN 'f' THEN 1 END) days_unavailable
	FROM
		calendar
	GROUP BY 1
)

SELECT

	neighbourhood,
	SUM(CAST(number_of_reviews as INTEGER)) total_reviews,
	AVG(CAST(review_scores_rating as INTEGER)) average_review,
	SUM(availability.total_days) total_days,
	SUM(availability.days_unavailable) days_unavailable,
	CAST(SUM(availability.days_unavailable) as float) / SUM(availability.total_days) * 100 percent_unavailable
FROM

	availability
JOIN

	listings1    
ON

	availability.listing_id = listings1.id
GROUP BY 1

ORDER BY total_reviews DESC

Capitol Hill is the most popular neighborhood having the most reviews at an average of 70% review score. It also has a comparatively high percent of unavailable listings (62%), which imnplies that customers tend to book in this neighborhood, leading to fewer available listings.

### When is the cheapest time to go to your city? When is the busiest?

#### Cheapest:

SELECT

	DATE(date) date,
	AVG(CAST(LTRIM(price,'$') as REAL)) average_price    
FROM

	calendar    
GROUP BY 1

ORDER BY 2

#### Busiest:

SELECT

	DATE(date) date,
	COUNT(listing_id) total_listings,
	COUNT(CASE available WHEN 'f' THEN 1 END) unavailable_listings,
	CAST(COUNT(CASE available WHEN 'f' THEN 1 END) as FLOAT) / COUNT(listing_id) * 100 percent_unavailable    
FROM

	calendar    
GROUP BY 1

ORDER BY 3 DESC